In [1]:
from pymongo import MongoClient
import pandas as pd 
import numpy as np

In [2]:
from pandas.io.json import json_normalize

In [111]:
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM

In [107]:
import folium
from folium import plugins

In [3]:
client=MongoClient()
db=client.companies
colec=db.companies

In [70]:
query1=colec.find({"$and":[ {'category_code': 'web'},
                            {'offices.country_code':'USA'},
                            {'offices':{'$size':1}}]},
                             {'name':1,'offices':1})
#len(list(query1)) #1736

In [71]:
df=pd.DataFrame(query1)
df.head(10)

,_id,name,offices
0,52cdef7c4bab8bd675297d90,Postini,"[{'description': None, 'address1': '959 Skyway..."
1,52cdef7c4bab8bd675297d91,Geni,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d93,Fox Interactive Media,"[{'description': '', 'address1': '407 N Maple ..."
3,52cdef7c4bab8bd675297d96,Gizmoz,"[{'description': None, 'address1': None, 'addr..."
4,52cdef7c4bab8bd675297d9b,eBay,"[{'description': 'Headquarters', 'address1': '..."
5,52cdef7c4bab8bd675297d9f,Viacom,"[{'description': 'HQ', 'address1': '1515 Broad..."
6,52cdef7c4bab8bd675297da1,Plaxo,"[{'description': 'HQ', 'address1': '1050 Enter..."
7,52cdef7c4bab8bd675297da3,Yahoo!,"[{'description': None, 'address1': '701 First ..."
8,52cdef7c4bab8bd675297da9,Mahalo,"[{'description': '', 'address1': '3525 Eastham..."
9,52cdef7c4bab8bd675297dad,Gannett,"[{'description': None, 'address1': '7950 Jones..."


In [72]:
df.shape

(1736, 3)

In [33]:
flat_json=json_normalize(df.offices[3])

flat_json

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.39417


In [73]:
def flatten(data, cols):
    for c in cols:
        fl=pd.DataFrame(dict(data[c])).T
        cols=[str(col) for col in fl.columns]
        fl.columns=[c+'_'+name for name in cols]
        data=pd.concat([data, fl], axis=1)
        data=data.drop(c, axis=1)
    return data

In [74]:
nested_cols=['offices']

flat=flatten(df, nested_cols)

flat.head()

,_id,name,offices_0
0,52cdef7c4bab8bd675297d90,Postini,"{'description': None, 'address1': '959 Skyway ..."
1,52cdef7c4bab8bd675297d91,Geni,"{'description': 'Headquarters', 'address1': '9..."
2,52cdef7c4bab8bd675297d93,Fox Interactive Media,"{'description': '', 'address1': '407 N Maple D..."
3,52cdef7c4bab8bd675297d96,Gizmoz,"{'description': None, 'address1': None, 'addre..."
4,52cdef7c4bab8bd675297d9b,eBay,"{'description': 'Headquarters', 'address1': '2..."


In [75]:
res=flat.copy()

total=flatten(res, ['offices_0'])
total.head(10)

,_id,name,offices_0_description,offices_0_address1,offices_0_address2,offices_0_zip_code,offices_0_city,offices_0_state_code,offices_0_country_code,offices_0_latitude,offices_0_longitude
0,52cdef7c4bab8bd675297d90,Postini,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.5069,-122.248
1,52cdef7c4bab8bd675297d91,Geni,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.0904,-118.393
2,52cdef7c4bab8bd675297d93,Fox Interactive Media,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.0762,-118.394
3,52cdef7c4bab8bd675297d96,Gizmoz,None,None,None,None,Menlo Park,CA,USA,37.4841,-122.169
4,52cdef7c4bab8bd675297d9b,eBay,Headquarters,2145 Hamilton Avenue,,95125,San Jose,CA,USA,37.295,-121.93
5,52cdef7c4bab8bd675297d9f,Viacom,HQ,1515 Broadway,,10036,New York,NY,USA,40.7577,-73.986
6,52cdef7c4bab8bd675297da1,Plaxo,HQ,"1050 Enterprise Way, 5th floor",,94089,Sunnyvale,CA,USA,37.3878,-122.055
7,52cdef7c4bab8bd675297da3,Yahoo!,None,701 First Avenue,,94089,Sunnyvale,CA,USA,37.4185,-122.025
8,52cdef7c4bab8bd675297da9,Mahalo,,3525 Eastham Dr.,,90232,Culver City,CA,USA,34.0176,-118.487
9,52cdef7c4bab8bd675297dad,Gannett,None,7950 Jones Branch Drive,,22107,McLean,VA,USA,38.9293,-77.2166


In [80]:
webNY=total[total['offices_0_state_code']=='NY'].copy()
webNY=webNY.drop('_id',axis=1)
webNY.head()

,name,offices_0_description,offices_0_address1,offices_0_address2,offices_0_zip_code,offices_0_city,offices_0_state_code,offices_0_country_code,offices_0_latitude,offices_0_longitude
5,Viacom,HQ,1515 Broadway,,10036,New York,NY,USA,40.7577,-73.986
13,Meetup,None,632 Broadway,10th Floor,10012,New York,NY,USA,40.726,-73.9957
75,Forbes,,60 5th Avenue,,10011,New York,NY,USA,40.7364,-73.9933
87,AOL,HQ,770 Broadway,,10003,New York,NY,USA,40.7311,-73.9919
93,Zwinky,,"One North Lexington, 9th Floor",White Plains,"NY, 10601",New York,NY,USA,None,None


In [86]:
query2=colec.find({"$and":[ {'category_code': 'ecommerce'},
                            {'offices.country_code':'USA'},
                            {'offices':{'$size':1}}]},
                             {'name':1,'offices':1})
#len(list(query2)) 295 elementos

In [87]:
df2=pd.DataFrame(query2)
df2.head()

,_id,name,offices
0,52cdef7c4bab8bd675297dd0,Wize,"[{'description': '', 'address1': '1110 Burling..."
1,52cdef7c4bab8bd675297dff,TheFind,"[{'description': '', 'address1': '310 Villa St..."
2,52cdef7c4bab8bd675297e01,Zazzle,"[{'description': '', 'address1': '1800 Seaport..."
3,52cdef7c4bab8bd675297e16,Edgeio,"[{'description': None, 'address1': '654 High S..."
4,52cdef7c4bab8bd675297e17,Kaboodle,"[{'description': '', 'address1': '640 W. Calif..."


In [88]:
flat2=flatten(df2, nested_cols)
res2=flat2.copy()
total2=flatten(res2, ['offices_0'])
total2.head(10)

,_id,name,offices_0_description,offices_0_address1,offices_0_address2,offices_0_zip_code,offices_0_city,offices_0_state_code,offices_0_country_code,offices_0_latitude,offices_0_longitude
0,52cdef7c4bab8bd675297dd0,Wize,,1110 Burlingame Ave,Suite 300,94010,Burlingame,CA,USA,37.5593,-122.302
1,52cdef7c4bab8bd675297dff,TheFind,,310 Villa Street,,94041,Mountain View,CA,USA,37.3915,-122.073
2,52cdef7c4bab8bd675297e01,Zazzle,,1800 Seaport,,94063,Redwood City,CA,USA,37.5109,-122.201
3,52cdef7c4bab8bd675297e16,Edgeio,None,654 High Street,#200,94301,Palo Alto,CA,USA,37.4431,-122.162
4,52cdef7c4bab8bd675297e17,Kaboodle,,640 W. California Ave.,Suite 220,94041,Sunnyvale,CA,USA,37.3822,-122.036
5,52cdef7c4bab8bd675297e66,Wishpot,Headquarters,911 Western Ave.,Suite 420,98104,Seattle,WA,USA,47.6207,-122.348
6,52cdef7c4bab8bd675297e74,Hotswap,None,None,None,None,Berkeley,CA,USA,37.8722,-122.268
7,52cdef7c4bab8bd675297e7a,Amazon,None,1200 12th Ave,S # 1200,98144,Seattle,WA,USA,47.5923,-122.317
8,52cdef7c4bab8bd675297eac,Viagogo,,77 Battery St Fl 4,,94111-5540,San Francisco,CA,USA,51.5002,-0.126236
9,52cdef7c4bab8bd675297ede,Bountii,,,,,Palo Alto,CA,USA,37.4373,-122.16


In [90]:
ecomNY=total2[total2['offices_0_state_code']=='NY'].copy()
ecomNY=ecomNY.drop('_id',axis=1)
ecomNY.head()

,name,offices_0_description,offices_0_address1,offices_0_address2,offices_0_zip_code,offices_0_city,offices_0_state_code,offices_0_country_code,offices_0_latitude,offices_0_longitude
21,ideeli,,620 8th Ave,45th Floor,10018,New York,NY,USA,40.756,-73.9904
22,Etsy,,55 Washington St.,5th Floor,11201,Brooklyn,NY,USA,40.6947,-73.9833
24,TableXchange,None,None,None,None,New York City,NY,USA,40.7579,-73.9855
47,Bonobos,,45 West 25th Street,5th Floor,10011,New York,NY,USA,40.7385,-73.9955
60,StubOrder,None,100 Maple Street,,10004,New York,NY,USA,40.6604,-73.9576


In [93]:
webecom=pd.concat([webNY, ecomNY])
webecom.head()

,name,offices_0_description,offices_0_address1,offices_0_address2,offices_0_zip_code,offices_0_city,offices_0_state_code,offices_0_country_code,offices_0_latitude,offices_0_longitude
5,Viacom,HQ,1515 Broadway,,10036,New York,NY,USA,40.7577,-73.986
13,Meetup,None,632 Broadway,10th Floor,10012,New York,NY,USA,40.726,-73.9957
75,Forbes,,60 5th Avenue,,10011,New York,NY,USA,40.7364,-73.9933
87,AOL,HQ,770 Broadway,,10003,New York,NY,USA,40.7311,-73.9919
93,Zwinky,,"One North Lexington, 9th Floor",White Plains,"NY, 10601",New York,NY,USA,None,None


In [95]:
webecom=webecom.drop('offices_0_description',axis=1)


(232, 10)

In [103]:
mapa=webecom.copy()

In [105]:
mapa=mapa.dropna(subset=['offices_0_latitude', 'offices_0_longitude'])

In [106]:
mapa=mapa[['name', 'offices_0_latitude', 'offices_0_longitude']]
mapa.head()

,name,offices_0_latitude,offices_0_longitude
5,Viacom,40.7577,-73.986
13,Meetup,40.726,-73.9957
75,Forbes,40.7364,-73.9933
87,AOL,40.7311,-73.9919
114,Veotag,40.753,-73.9855


In [129]:
def generateBaseMap(default_location=[40.712784, -74.005941], default_zoom_start=9):
    
    base_map = folium.Map(location=default_location, 
                          control_scale=True, 
                          zoom_start=default_zoom_start)
    
    return base_map

In [130]:
mapaprueba=generateBaseMap()

In [131]:
HM(data=mapa[['offices_0_latitude', 
                 'offices_0_longitude']], 
    radius=14, 
    max_zoom=13).add_to(mapaprueba) # pon los datos en heatmap

mapaprueba.save('mapaprueba.html')  # guarda

mapaprueba   # representa